In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import gzip

# keras framework 
from keras.datasets import mnist
from keras.layers import Dense, Flatten, Reshape
from keras.layers.advanced_activations import LeakyReLU, ReLU
from keras.models import Sequential
from keras.optimizers import Adam

Using TensorFlow backend.


In [2]:
# image dimensions
img_rows = 28
img_cols = 28
channels = 1
img_shape = (img_rows, img_cols, channels) # 28 x 28 x 1

z_dim = 100 # size of the noise vector

In [3]:
# a few helper functions
def extract_data(filename, num_images):
    # open the zip file as a stream
    with gzip.open(filename) as bytestream:
        # read in 16 bytes 
        bytestream.read(16)
        buf = bytestream.read(28 * 28 * num_images)
        data = np.frombuffer(buf, dtype=np.uint8).astype(np.float32)
        data = data.reshape(num_images, 28,28)
        return data

# read in train data ... 60,000 images
#train_data = extract_data('train-images-idx3-ubyte.gz', 60000)
# read in test data ... 10,000 images
train_data = extract_data('t10k-images-idx3-ubyte.gz', 10000)

In [4]:
# build generator ... a neural network with one hidden layer
def build_generator(img_shape, z_dim):
    model = Sequential() # stack the layers

    model.add(Dense(128, input_dim=z_dim))

    model.add(LeakyReLU(alpha=0.01))
    #model.add(ReLU(alpha=0.01)) # use ReLU activation with 0.01 learning rate

    model.add(Dense(28 * 28 * 1, activation='tanh')) # transfer function .. tanh => this yiles ouput in [-1, 1]
    # tanh instead of sigmoid...research & empirical results support tanh as a better funtion

    model.add(Reshape(img_shape)) # reshape the output to represent the original image dimensions

    return model



In [5]:
# build the discriminator .. a neural network with a single layer
def build_discriminator(img_shape):

    model = Sequential()

    model.add(Flatten(input_shape=img_shape))

    model.add(Dense(128))

    model.add(LeakyReLU(alpha=0.01))
    #model.add(ReLU(alpha=0.01))

    model.add(Dense(1, activation='sigmoid')) # sigmoid instead of tanh because we want to out a probability value

    return model



In [6]:
def build_gan(generator, discriminator):

    model = Sequential()

    model.add(generator)
    model.add(discriminator)

    return model


discriminator = build_discriminator(img_shape)

# this fixes the discriminator parameters, allowing the model to temporaily train only the generator
#discriminator.trainable = False  

discriminator.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

generator = build_generator(img_shape, z_dim)

# this fixes the discriminator parameters, allowing the model to temporaily train only the generator
#discriminator.trainable = False  

gan = build_gan(generator, discriminator)

gan.compile(loss='binary_crossentropy', optimizer=Adam()) # cross entropy loss/ log loss

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [7]:
losses = []
accuracies = []
iteration_checkpoints = []

def train(iterations, batch_size, sample_interval, train_data):

    (X_train, _), (_, _) = mnist.load_data()
    #train_data = train_data / 127.5 - 1.0
    X_train = X_train / 127.5 - 1.0# train_data  
    X_train = np.expand_dims(X_train, axis=3)

    real = np.ones((batch_size, 1))

    fake = np.zeros((batch_size, 1))

    for iteration in range(iterations):



        idx = np.random.randint(0, X_train.shape[0], batch_size)
        imgs = X_train[idx]

        z = np.random.normal(0, 1, (batch_size, 100))
        gen_imgs = generator.predict(z)

        d_loss_real = discriminator.train_on_batch(imgs, real)
        d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
        d_loss, accuracy = 0.5 * np.add(d_loss_real, d_loss_fake)



        z = np.random.normal(0, 1, (batch_size, 100))
        gen_imgs = generator.predict(z)

        g_loss = gan.train_on_batch(z, real)

        if (iteration + 1) % sample_interval == 0:

            losses.append((d_loss, g_loss))
            accuracies.append(100.0 * accuracy)
            iteration_checkpoints.append(iteration + 1)

            print("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" %
                  (iteration + 1, d_loss, 100.0 * accuracy, g_loss))

            sample_images(generator)

In [8]:
def sample_images(generator, image_grid_rows=4, image_grid_columns=4):

    z = np.random.normal(0, 1, (image_grid_rows * image_grid_columns, z_dim))

    gen_imgs = generator.predict(z)

    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(image_grid_rows,
                            image_grid_columns,
                            figsize=(4, 4),
                            sharey=True,
                            sharex=True)

    cnt = 0
    for i in range(image_grid_rows):
        for j in range(image_grid_columns):
            axs[i, j].imshow(gen_imgs[cnt, :, :, 0], cmap='gray')
            axs[i, j].axis('off')
            cnt += 1
    
    #plt.figure(figsize=[5,5])

    # Display the first image in training data
    #plt.subplot(121)
    # get the image
    #curr_img = np.reshape(train_data[0], (28,28))
    # get the train label
    #curr_lbl = train_labels[0]
    #plt.imshow(gen_imgs[1, :, :, 0], cmap='gray')
    #plt.axis('off')
    #plt.title("(Label: " + str(label_dict[curr_lbl]) + ")")


In [ ]:
iterations = 20000
batch_size = 128
sample_interval = 1000

train(iterations, batch_size, sample_interval, train_data)

In [34]:
(X_train, _), (_, _) = mnist.load_data()
np.max(X_train)

255